In [13]:
# Deliverable 1: Retrieve Weather Data
# -------------------------------------------------------------------
# Generate a set of 2,000 random latitudes and longitudes, retrieve the nearest city, and perform an API call 
# with the OpenWeatherMap. In addition to the city weather data you gathered in this module, use your API skills 
# to retrieve the current weather description for each city. Then, create a new DataFrame containing the updated 
# weather data.


import numpy as np

from citipy import citipy

from config_weather import weather_api_key 

import requests

import time

import pandas as pd

In [2]:
#Create a new set of 2,000 random latitudes and longitudes.

lat = np.random.uniform(-90,90, 2000)
lng = np.random.uniform(-180,180, 2000)

lat_lng = zip(lat,lng) #pairs the array

coordinates = list(lat_lng) #unzipping zipped tuple into a list

coordinates

[(-50.615315383689996, 38.16572697015249),
 (76.49148430815833, -42.23625575976692),
 (70.33350546933801, -15.633176461951734),
 (-37.24004441766157, 23.63499811652619),
 (-72.6908269291807, 54.965737980682746),
 (-65.87369609017783, 7.963293091652162),
 (-71.85571702013411, 166.63058090842583),
 (-16.456037868415933, -145.25679515337663),
 (-55.20972673665555, 133.69613799085016),
 (-10.832696419545428, -33.282171033057494),
 (88.63626961240382, 156.66477844171732),
 (13.890257429510655, -76.06791133445016),
 (-59.5757036274273, -146.61579924114574),
 (-36.181129314412274, -168.39266083806288),
 (-37.45001470240935, -83.44155924771322),
 (-20.374965215130587, 22.332454038571825),
 (73.89952123324622, -66.9637357873349),
 (67.30881971159423, 108.72182629640423),
 (46.80842573983071, -101.3197060853201),
 (-47.880934184726776, 141.4423245363846),
 (-29.154240351298753, -88.66976363117237),
 (-49.940138919875196, 160.70723786234646),
 (72.22116647234412, -49.414212794313016),
 (20.592782

In [3]:
#Get the nearest city using the citipy module.

cities = []

for row in coordinates:
    city = citipy.nearest_city(row[0], row[1]).city_name #row[0] is lat, #row[1] is long
    if city not in cities:
        cities.append(city)
    
cities


['east london',
 'ilulissat',
 'husavik',
 'plettenberg bay',
 'hermanus',
 'bluff',
 'tautira',
 'new norfolk',
 'maragogi',
 'chokurdakh',
 'puerto colombia',
 'mataura',
 'vaini',
 'lebu',
 'sehithwa',
 'clyde river',
 'aykhal',
 'mandan',
 'tuatapere',
 'shenzhen',
 'klaksvik',
 'zhigansk',
 'port macquarie',
 'grindavik',
 'saskylakh',
 'san martin',
 'taolanaro',
 'kapaa',
 'nikolayevka',
 'kupang',
 'igrim',
 'mar del plata',
 'port alfred',
 'tuktoyaktuk',
 'margate',
 'hasaki',
 'avarua',
 'albany',
 'monmouth',
 'nizhneyansk',
 'namibe',
 'saldanha',
 'atuona',
 'srednekolymsk',
 'dunedin',
 'mokhsogollokh',
 'hobart',
 'nikolskoye',
 'garowe',
 'avera',
 'namanga',
 'castro',
 'georgetown',
 'yellowknife',
 'tasiilaq',
 'hilo',
 'petrolina de goias',
 'qaanaaq',
 'butaritari',
 'waddan',
 'mocuba',
 'rikitea',
 'nosy varika',
 'san vicente',
 'ushuaia',
 'puerto ayora',
 'havoysund',
 'saint george',
 'cururupu',
 'punta arenas',
 'grand river south east',
 'kargalinskaya',


In [11]:
# Perform an API call with the OpenWeatherMap.



#empty list to hold weather data
city_data = []

#print beginning of logging
print("Beginning of retrieval    ")
print("----------------------------")

#create counters
record_count = 1
set_count = 1

#loop through all cities
for i, city in enumerate(cities): # this lets us have both index and value #range(len(cities)) gives just index, for city in cities gives just city:
    #go through cities 50 at a time for logging purposes
    if (i % 50 == 0 and i >= 50): 
        set_count +=1
        record_count = 1 
        time.sleep(60) #tells program to stop for 60 seconds because API only lets us have 60 calls per minute on the free version to prevent time out errors
    #new city url for each city
    city_url = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid=" + weather_api_key + "&units=imperial" + "&q=" + city.replace(" ","+") #some city names have spaces, so we're taking them out and concatinating the names for the URL (otherwise it will find data for just the first part of the city name)
    print (f"processing record {record_count} of set {set_count} | {city}")
    #add 1 to record count
    record_count +=1

    #run API request on all the cities:
    try: 
        # Parse the JSON and retrieve data.
        weather_request=requests.get(city_url).json()
        
        #Parse out the needed data
        latitude = weather_request["coord"]["lon"]
        longitude = weather_request["coord"]["lat"]
        max_temp = weather_request["main"]["temp_max"]
        humidity = weather_request["main"]["humidity"]
        cloudiness = weather_request["clouds"]["all"]
        wind_speed = weather_request["wind"]["speed"]
        weather_description = weather_request["weather"][0]["description"] #weather is a list of dictionaries, so look at the first row in the list (at index 0), and then the description key, with the "clear sky" value
        
        #adding data to city data list:
        city_data.append({ #appending data as a list of dictionaries
            "city":city.title(), #.title makes the first letter in each string capitalized
            "latitude":latitude,
            "longitude":longitude,
            "max_temp":max_temp,
            "humidity":humidity,
            "cloudiness":cloudiness,
            "wind_speed":wind_speed,
            "weather_description":weather_description})
    #if an error comes up, skip the city
    except:
        print("City not found. Skipping...")
        pass #tells it that if city is not found, don't add previous city's data #pass is a general purpose statement to handle all errors encountered and to allow the program to continue
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    


Beginning of retrieval    
----------------------------
processing record 1 of set 1 | east london
processing record 2 of set 1 | ilulissat
processing record 3 of set 1 | husavik
processing record 4 of set 1 | plettenberg bay
processing record 5 of set 1 | hermanus
processing record 6 of set 1 | bluff
processing record 7 of set 1 | tautira
processing record 8 of set 1 | new norfolk
processing record 9 of set 1 | maragogi
processing record 10 of set 1 | chokurdakh
processing record 11 of set 1 | puerto colombia
processing record 12 of set 1 | mataura
processing record 13 of set 1 | vaini
processing record 14 of set 1 | lebu
processing record 15 of set 1 | sehithwa
processing record 16 of set 1 | clyde river
processing record 17 of set 1 | aykhal
processing record 18 of set 1 | mandan
processing record 19 of set 1 | tuatapere
processing record 20 of set 1 | shenzhen
processing record 21 of set 1 | klaksvik
processing record 22 of set 1 | zhigansk
processing record 23 of set 1 | port macq

processing record 38 of set 4 | phra phutthabat
processing record 39 of set 4 | karauzyak
City not found. Skipping...
processing record 40 of set 4 | torbay
processing record 41 of set 4 | luanda
processing record 42 of set 4 | bethel
processing record 43 of set 4 | ostrovnoy
processing record 44 of set 4 | derzhavinsk
processing record 45 of set 4 | kutum
processing record 46 of set 4 | kailua
processing record 47 of set 4 | faanui
processing record 48 of set 4 | illoqqortoormiut
City not found. Skipping...
processing record 49 of set 4 | sandwick
processing record 50 of set 4 | sarakhs
processing record 1 of set 5 | trinidad
processing record 2 of set 5 | rondonopolis
processing record 3 of set 5 | salalah
processing record 4 of set 5 | balkhash
processing record 5 of set 5 | gallup
processing record 6 of set 5 | zaysan
processing record 7 of set 5 | thunder bay
processing record 8 of set 5 | bielsk podlaski
processing record 9 of set 5 | tiksi
processing record 10 of set 5 | batagay

processing record 27 of set 8 | marsh harbour
processing record 28 of set 8 | bengkulu
processing record 29 of set 8 | atambua
processing record 30 of set 8 | darnah
processing record 31 of set 8 | bolzano
processing record 32 of set 8 | yumen
processing record 33 of set 8 | kaitangata
processing record 34 of set 8 | la union
processing record 35 of set 8 | mouzakion
City not found. Skipping...
processing record 36 of set 8 | tumannyy
City not found. Skipping...
processing record 37 of set 8 | bonfim
processing record 38 of set 8 | palmer
processing record 39 of set 8 | orlik
processing record 40 of set 8 | awbari
processing record 41 of set 8 | waynesville
processing record 42 of set 8 | piacabucu
processing record 43 of set 8 | dickson
processing record 44 of set 8 | fare
processing record 45 of set 8 | katsuura
processing record 46 of set 8 | samarai
processing record 47 of set 8 | haimen
processing record 48 of set 8 | vaitape
processing record 49 of set 8 | aquiraz
processing reco

processing record 13 of set 12 | savannah bight
processing record 14 of set 12 | kahului
processing record 15 of set 12 | pasighat
processing record 16 of set 12 | ribeira brava
processing record 17 of set 12 | lidkoping
processing record 18 of set 12 | aichach
processing record 19 of set 12 | flin flon
processing record 20 of set 12 | raga
processing record 21 of set 12 | ereymentau
processing record 22 of set 12 | carauari
processing record 23 of set 12 | pisco
processing record 24 of set 12 | marcona
City not found. Skipping...
processing record 25 of set 12 | ancud
processing record 26 of set 12 | adrian
processing record 27 of set 12 | medicine hat
processing record 28 of set 12 | okhotsk
processing record 29 of set 12 | hambantota
processing record 30 of set 12 | longyearbyen
processing record 31 of set 12 | san nicola
processing record 32 of set 12 | trofors
processing record 33 of set 12 | chapais
processing record 34 of set 12 | skagastrond
City not found. Skipping...
processi

processing record 45 of set 15 | oksfjord
processing record 46 of set 15 | ruatoria
City not found. Skipping...
processing record 47 of set 15 | dzaoudzi
processing record 48 of set 15 | huaiyin
processing record 49 of set 15 | andilamena
processing record 50 of set 15 | dedovichi
processing record 1 of set 16 | veraval
processing record 2 of set 16 | soldato-aleksandrovskoye
processing record 3 of set 16 | puerto gaitan
processing record 4 of set 16 | coihaique
processing record 5 of set 16 | mbini
processing record 6 of set 16 | raska
processing record 7 of set 16 | mazulskiy
City not found. Skipping...
processing record 8 of set 16 | stornoway
processing record 9 of set 16 | bargal
City not found. Skipping...
processing record 10 of set 16 | kawambwa
processing record 11 of set 16 | alugan
processing record 12 of set 16 | sumbawa
City not found. Skipping...
processing record 13 of set 16 | locri
processing record 14 of set 16 | flinders
-----------------------------
Data Retrieval C

In [8]:
city_data

[{'city': 'East London',
  'latitude': 27.9116,
  'longitude': -33.0153,
  'max_temp': 52.74,
  'humidity': 76,
  'cloudiness': 0,
  'wind_speed': 6.91,
  'weather_description': 'clear sky'},
 {'city': 'Ilulissat',
  'latitude': -51.1,
  'longitude': 69.2167,
  'max_temp': 26.46,
  'humidity': 62,
  'cloudiness': 3,
  'wind_speed': 8.39,
  'weather_description': 'clear sky'},
 {'city': 'Husavik',
  'latitude': -17.3389,
  'longitude': 66.0449,
  'max_temp': 48.09,
  'humidity': 77,
  'cloudiness': 83,
  'wind_speed': 3.96,
  'weather_description': 'broken clouds'}]

In [15]:
#add weather to a new dataframe
city_df = pd.DataFrame(city_data)

city_df


,city,latitude,longitude,max_temp,humidity,cloudiness,wind_speed,weather_description
0,East London,27.9116,-33.0153,52.74,76,0,6.91,clear sky
1,Ilulissat,-51.1000,69.2167,33.82,69,3,2.30,clear sky
2,Husavik,-17.3389,66.0449,48.09,77,83,3.96,broken clouds
3,Plettenberg Bay,23.3716,-34.0527,62.74,42,1,4.09,clear sky
4,Hermanus,19.2345,-34.4187,63.68,42,44,8.50,scattered clouds
...,...,...,...,...,...,...,...,...
694,Stornoway,-6.3865,58.2093,51.76,84,20,4.61,few clouds
695,Kawambwa,29.0791,-9.7915,52.14,76,86,7.67,overcast clouds
696,Alugan,125.4808,12.2188,84.25,72,100,7.70,overcast clouds
697,Locri,16.2623,38.2414,75.33,80,0,13.09,clear sky


In [16]:
#Export the DataFrame as WeatherPy_Database.csv into the Weather_Database folder

file_name="WeatherPy_Database.csv"

city_df.to_csv(file_name, index="city_id")